In [1]:
# Import libraries
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from utils.database import get_db
import utils.models as models
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Create connection to Database and extraction of required data

In [2]:
# create connection to the database
db = get_db()

In [3]:
# Define requested ticker ID:
ticker_id =

In [4]:
# set the presentation format of float numbers within the dataframe
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Fetch "daily multipliers" table from database

In [ ]:
query = db.query(models.FullDailyMultipliers)
df_daily_multipliers = pd.read_sql(query.statement, query.session.bind)

In [ ]:
# check the table was retrieved
df_daily_multipliers.head()

In [ ]:
# Select the stock according to given stock id (= ticker_id)
daily_multipliers = df_daily_multipliers.loc[df_daily_multipliers['stock_id'] == ticker_id]

# Delete all dates < 2.1.13
daily_multipliers = daily_multipliers.query('"2013-01-01" < date').copy().reset_index(drop=True)

# Check the table is filtered correctly
daily_multipliers.head()

In [ ]:
# Check for missing values in table
df_daily_multipliers[df_daily_multipliers.isna().any(axis=1)]

# In case there are missing values in the table, run the following preprocessing code (otherwise - can skip):

In [ ]:
# Check for null values - if found - complete null values using interpolation (up to 5 days in a row)

# First, find Nan values in dataframe, save this as a mask:
mask = daily_multipliers.isna()

# Then, calculate the length of consecutive Nans in order to find Nans > 5, appoint each Nan that's found with its defining "score" :
df_na_sizes = (mask.ne(mask.shift()).cumsum()
                   .where(mask)
                   .apply(lambda c: c.groupby(c).transform('size'))
               )

# In order to differentiate which rows have Nan values should be erased - rename the Nan values with score >=6 with the word "Drop":
df_without_consec_nan = daily_multipliers.mask(df_na_sizes.ge(6), 'drop')

In [ ]:
# Next, erase all rows that have a "Drop" value in them:
df_filtered = df_without_consec_nan[(df_without_consec_nan.iloc[:, 3:] != 'drop').all(axis=1)]

df_filtered

In [ ]:
# Finally, handle the remaining Nan values in dataframe by using interpolation:
df_filtered = df_filtered.astype({"trailing_peg_1_y" : float})

df_filtered.iloc[:, 3:] = df_filtered.iloc[:, 3:].interpolate(axis=0, limit_area='inside', limit=5)

daily_multipliers = df_filtered

In [ ]:
# Check to make sure there are no Nan values in final dataframe
daily_multipliers[daily_multipliers.isna().any(axis=1)]

### Fetch "end of day prices" table from database

In [ ]:
query = db.query(models.EndOfDayPrices)
df_end_of_day_prices = pd.read_sql(query.statement, query.session.bind)

In [ ]:
# check the table was retrieved
df_end_of_day_prices.head()

### Select specific stock from "end of day prices" table

In [ ]:
# Select the stock according to given stock id (= ticker_id)
end_of_day_prices = df_end_of_day_prices.loc[df_end_of_day_prices['stock_id'] == ticker_id]

# Delete all dates < 2.1.13
end_of_day_prices = end_of_day_prices.query('"2013-01-01" < date').copy().reset_index(drop=True)

# Check the table is filtered correctly
end_of_day_prices.head()

In [ ]:
# Check for missing values in table
end_of_day_prices[end_of_day_prices.isna().any(axis=1)]

# In case there are missing values in the table, run the following preprocessing code (otherwise - can skip):

In [ ]:
# Check for null values - if found - complete null values using interpolation (up to 5 days in a row)

# First, find Nan values in dataframe, save this as a mask:
mask = end_of_day_prices.isna()

# Then, calculate the length of consecutive Nans in order to find Nans > 5, appoint each Nan that's found with its defining "score" :
df_na_sizes = (mask.ne(mask.shift()).cumsum()
                   .where(mask)
                   .apply(lambda c: c.groupby(c).transform('size'))
               )

# In order to differentiate which rows have Nan values should be erased - rename the Nan values with score >=6 with the word "Drop":
df_without_consec_nan = end_of_day_prices.mask(df_na_sizes.ge(6), 'drop')

In [ ]:
# Next, erase all rows that have a "Drop" value in them:
df_filtered = df_without_consec_nan[(df_without_consec_nan.iloc[:, 3:] != 'drop').all(axis=1)]

df_filtered

In [ ]:
# Finally, handle the remaining Nan values in dataframe by using interpolation:
df_filtered = df_filtered.astype({"trailing_peg_1_y" : float})

df_filtered.iloc[:, 3:] = df_filtered.iloc[:, 3:].interpolate(axis=0, limit_area='inside', limit=5)

end_of_day_prices = df_filtered

In [ ]:
# Check to make sure there are no Nan values in final dataframe
end_of_day_prices[end_of_day_prices.isna().any(axis=1)]

### Fetch "pfree cash flow multiplier" table from database

In [ ]:
query = db.query(models.PFreeCashFlowMultiplier)
df_pfree_cash_flow = pd.read_sql(query.statement, query.session.bind)

In [ ]:
# check the table was retrieved
df_end_of_day_prices.head()

### Select specific stock from "pfree cash flow multiplier" table

In [ ]:
# Select the stock according to given stock id (= ticker_id)
pfree_cash_flow = df_pfree_cash_flow.loc[df_pfree_cash_flow['stock_id'] == ticker_id]

# Delete all dates < 2.1.13
pfree_cash_flow = pfree_cash_flow.query('"2013-01-01" < date').copy().reset_index(drop=True)

# Check the table is filtered correctly
pfree_cash_flow.head()

In [ ]:
# Check for missing values in table
pfree_cash_flow[pfree_cash_flow.isna().any(axis=1)]

# In case there are missing values in the table, run the following preprocessing code (otherwise - can skip):

In [ ]:
# Check for null values - if found - complete null values using interpolation (up to 5 days in a row)

# First, find Nan values in dataframe, save this as a mask:
mask = pfree_cash_flow.isna()

# Then, calculate the length of consecutive Nans in order to find Nans > 5, appoint each Nan that's found with its defining "score" :
df_na_sizes = (mask.ne(mask.shift()).cumsum()
                   .where(mask)
                   .apply(lambda c: c.groupby(c).transform('size'))
               )

# In order to differentiate which rows have Nan values should be erased - rename the Nan values with score >=6 with the word "Drop":
df_without_consec_nan = pfree_cash_flow.mask(df_na_sizes.ge(6), 'drop')

In [ ]:
# Next, erase all rows that have a "Drop" value in them:
df_filtered = df_without_consec_nan[(df_without_consec_nan.iloc[:, 3:] != 'drop').all(axis=1)]

df_filtered

In [ ]:
# Finally, handle the remaining Nan values in dataframe by using interpolation:

df_filtered = df_filtered.astype({"pfree_cash_flow_ratio" : float})

df_filtered.iloc[:, 3:] = df_filtered.iloc[:, 3:].interpolate(axis=0, limit_area='inside', limit=5)

pfree_cash_flow = df_filtered

In [ ]:
# Check if to make sure there are no Nan values in final dataframe
pfree_cash_flow[pfree_cash_flow.isna().any(axis=1)]

### Fetch "graham number" table from database

In [ ]:
query = db.query(models.GrahamNumber)
df_graham_number = pd.read_sql(query.statement, query.session.bind)

In [ ]:
# check the table was retrieved
df_graham_number.head()

### Select specific stock from "graham number" table

In [ ]:
# Select the stock according to given stock id (= ticker_id)
graham_number = df_graham_number.loc[df_graham_number['stock_id'] == ticker_id]

# filter year >= 2012
graham_number = graham_number.query('2012 < year').copy().reset_index(drop=True)

In [ ]:
# Check for null values - if found any - "graham number" dataframe will not be used in final joined dataframe
if graham_number['graham_value'].isnull().values.any():
    print("'Graham Number' dataframe will not be joined to final dataframe due to missing values")
else:
    print("'Graham Number' dataframe doesn't have missing values, can be joined to final dataframe")


### Fetch "overview data" table from database

In [5]:
overview = models.QuarterlyOverview
query = db.query(overview.stock_id, overview.year, overview.quarter, overview.currentRatio )
df_overview_data = pd.read_sql(query.statement, query.session.bind)

In [ ]:
# check the table was retrieved
df_overview_data.head()

In [7]:
# Choose the stock_id we want to check (ticker_id)
overview_data = df_overview_data.loc[df_overview_data['stock_id'] == ticker_id]

# filter out year =< 2012 and quarter "0"
overview_data = overview_data.query("2012 < year and quarter != 0").copy().reset_index(drop=True)

In [ ]:
# Check for null values - if found - "overeview data" dataframe will not be used in final joined dataframe
if overview_data['currentRatio'].isnull().values.any():
    print("'overeview data' dataframe will not be joined to final dataframe due to missing values")
else:
    print("'overeview data' dataframe doesn't have missing values, can be joined to final dataframe")


# Join all dataframes to one final dataframe

In [ ]:
# First, combine the two dataframes: "end of day prices" and "daily multipliers" into one joined dataframe (using outer join to maintain all rows)
joined_dataframe = pd.merge(end_of_day_prices, daily_multipliers, on='date', how='inner')

# Create 2 new columns for "date_plus_3m" & "market_cap_plus_3m"
day_gap = 60
date_plus_3m = joined_dataframe.loc[day_gap:,'date'].values
market_cap_plus_3m = joined_dataframe.loc[day_gap:,'market_cap'].values


joined_dataframe.drop(joined_dataframe.tail(day_gap).index, inplace = True)
joined_dataframe['date_plus_3m'] = date_plus_3m
joined_dataframe['market_cap_plus_3m'] = market_cap_plus_3m

# Rename columns
joined_dataframe.rename(columns = {'id_x': 'id', 'stock_id_x':'stock_id'}, inplace=True)

# Setting the index column to column 'id'
joined_dataframe['id'] = joined_dataframe.index

# Dropping redundant columns
joined_dataframe = joined_dataframe.drop(['stock_id_y', 'id_y'], axis=1)

# Rename dataframe for clarity purposes
multiplier_with_closing_prices = joined_dataframe

# Check for Nan values in joined dataframe
multiplier_with_closing_prices[multiplier_with_closing_prices.isna().any(axis=1)]

In [ ]:
# Second, join the two dataframes: "multiplier_with_closing_prices" and "pfree_cash_flow" into one joined dataframe (using outer join to maintain all rows)
joined_dataframe = pd.merge(multiplier_with_closing_prices, pfree_cash_flow, on='date', how='left')

# Rename columns
joined_dataframe.rename(columns = {'id_x': 'id', 'stock_id_x':'stock_id'}, inplace=True)

# Convert "date" column's type from object to datetime and "stock id" to int
joined_dataframe['date'] = pd.to_datetime(joined_dataframe['date'])
joined_dataframe['stock_id'] = joined_dataframe['stock_id'].astype('int')

# Handle Nan values that were formed due to the left join of the two dataframes
joined_dataframe["pfree_cash_flow_ratio"] = joined_dataframe["pfree_cash_flow_ratio"].interpolate(axis=0, limit_area='inside', limit=5)

joined_dataframe["year"] = joined_dataframe["date"].dt.year
joined_dataframe['quarter'] = joined_dataframe['date'].dt.quarter

# Setting the index column to column 'id'
joined_dataframe['id'] = joined_dataframe.index

# Dropping redundant columns
joined_dataframe = joined_dataframe.drop(['stock_id_y', 'id_y'], axis=1)

# Rename dataframe for clarity purposes
multiplier_with_closing_prices_and_cash_flow = joined_dataframe

# Check for Nan values in joined dataframe
multiplier_with_closing_prices_and_cash_flow[multiplier_with_closing_prices_and_cash_flow.isna().any(axis=1)]

### If "graham number" dataframe is without Nan - run the following cell, if not skip it

In [ ]:
# Next, merge the dataframe - "graham number", which has quarterly data with the previously joined dataframe,
joined_dataframe = pd.merge(multiplier_with_closing_prices_and_cash_flow, graham_number, on=['year', 'quarter'], how='left')

# Rename columns
joined_dataframe.rename(columns = {'id_x': 'id', 'stock_id_x':'stock_id'}, inplace=True)

# Dropping redundant columns
joined_dataframe = joined_dataframe.drop(['stock_id_y', 'id_y'], axis=1)

# Rename dataframe for clarity purposes
four_joined_dataframes = joined_dataframe

# Check for Nan values in joined dataframe
four_joined_dataframes[four_joined_dataframes.isna().any(axis=1)]

### If "overview" dataframe is without Nan - run one of the following possible cells, if not skip both of them

#### 1. Run the following cell if "graham number" was joined to final joined dataframe

In [ ]:
# Finally, merge the remaining dataframe - "overview", which has quarterly data with the previously joined dataframe,
joined_dataframe = pd.merge(four_joined_dataframes, overview_data, on=['stock_id', 'year', 'quarter'], how='left')

# Rename dataframe for clarity purposes
final_joined_dataframe = joined_dataframe

# Check for Nan values in joined dataframe
final_joined_dataframe[final_joined_dataframe.isna().any(axis=1)]

#### 2. Run the following cell if "graham number" was not joined to final joined dataframe

In [ ]:
# Finally, merge the remaining dataframe - "overview", which has quarterly data with the previously joined dataframe,
joined_dataframe = pd.merge(multiplier_with_closing_prices_and_cash_flow, overview_data, on=['stock_id', 'year', 'quarter'], how='left')

# Rename dataframe for clarity purposes
final_joined_dataframe = joined_dataframe

# Check for Nan values in joined dataframe
final_joined_dataframe[final_joined_dataframe.isna().any(axis=1)]

### Create new column: "diff_in_market_cap_perc"

In [ ]:
# create a new column - "diff_in_market_cap_perc" where:
# calculates the change in market cap value 3 months apart and presents the difference in percentage

col1 = final_joined_dataframe['market_cap']
col2 = final_joined_dataframe['market_cap_plus_3m']

final_joined_dataframe['diff_in_mc'] = col2.sub(col1, axis=0)
final_joined_dataframe['diff_in_mc_perc'] = ((col2.sub(col1, axis=0)).div(col1)).mul(100)

# Check to assure the final dataframe contains new column
final_joined_dataframe.head()

# Optimizing the random forest model

In [ ]:
# Define requested model and label thresholds:

label_threshold_lst = [1, 5, 10, 15, 20]

model_threshold_lst = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95]


In [ ]:
# First, convert "date_plus_3m" columns type from 'string' to 'datetime'
final_joined_dataframe['date_plus_3m'] = pd.to_datetime(final_joined_dataframe['date_plus_3m'])

# Split the data to 60% train, 20% validation, 20% test
train = final_joined_dataframe.query('"2013-01-01" < date <= "2017-03-30"').copy()
validation = final_joined_dataframe.query('"2017-03-30" < date <= "2018-08-29"').copy()
test = final_joined_dataframe.query('"2018-08-29" < date <= "2020-02-01"').copy()


# Set the x (=features)
X_Cols = ['enterprise_val','pe_ratio','pb_ratio','trailing_peg_1_y', 'pfree_cash_flow_ratio', 'graham_value', 'currentRatio']

# Split X to train and test
X_train, X_validation = train[X_Cols], validation[X_Cols]


In [ ]:
train.shape

In [ ]:
validation.shape

In [ ]:
test.shape

In [ ]:
# Model optimization using ParameterGrid() - ~percentage~ of profit

# Create empty dictionary that will hold all parameters tested
profit_with_params_dict = {}

# Define random forest model parameters to be examined
rf_clf_full_params = {
                'n_estimators':[51,101,151], # The number of trees in the forest
                'criterion':["gini", "entropy"], # The function to measure the quality of a split.
                'max_depth':[5, 7, 9, 11],    # The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples
                'max_features':[3, 5, 7, 8], # The number of features to consider when looking for the best split results
                'random_state':[711]
                 }

params = rf_clf_full_params
all_rf_params_permutations = list(ParameterGrid(params))


# Loop through different model parameters in order to find the optimized set of parameters for the requested stock, testing on a validation set:
for label_thresh in tqdm(label_threshold_lst, desc="label threshold"):
    train['label'] = (train["diff_in_mc_perc"] > label_thresh).astype(int)
    validation['label'] = (validation["diff_in_mc_perc"] > label_thresh).astype(int)
    Y_Cols = ['label']
    y_train, y_validation = train[Y_Cols].values.ravel(), validation[Y_Cols].values.ravel()  # this changes according to the tested label threshold, therefor inserted into the for loop
    for model_thresh in tqdm(model_threshold_lst, desc="model threshold"):
        for params in tqdm(all_rf_params_permutations, desc="model params"):
            rand_frst_clf = RandomForestClassifier(**params)  # run through the different classifier's parameters options
            rand_frst_clf.fit(X_train, y_train)   # fit (train) the data to the model
            y_pred_proba = rand_frst_clf.predict_proba(X_validation)[:,1] # Make predictions (validation)
            y_pred = y_pred_proba > model_thresh
            validation['prediction'] = y_pred.astype(int)
            validation['profit_percentage'] = validation['diff_in_mc_perc'] * validation['prediction']
            profit_perc_sum = round(validation['profit_percentage'].sum(), 3)
            profit_perc_mean = round(profit_perc_sum / validation[validation['prediction']==1].shape[0], 3)
            print('profit_perc_sum', profit_perc_sum, 'profit_perc_mean', profit_perc_mean, 'denominator', validation[validation['prediction']==1].shape[0])
            profit_with_params_dict[profit_perc_mean] = [label_thresh, model_thresh, params]

profit_keys_lst = list(profit_with_params_dict.keys())
profit_keys_lst_not_null = [i for i in profit_keys_lst if not pd.isna(i)]
profit_keys_lst_not_null.sort(reverse=True)
max_profit_perc_in_sorted_dict = {i: profit_with_params_dict[i] for i in profit_keys_lst_not_null[:5]}

print(max_profit_perc_in_sorted_dict)

# 2. Preparing necessary data for later model run on test / validation set (with reports)

In [ ]:
# extract best model parameters from dictionary
profit_keys_lst = list(profit_with_params_dict.keys())
profit_keys_lst_not_null = [i for i in profit_keys_lst if not pd.isna(i)]
profit_keys_lst_not_null.sort(reverse=True)
best_rf_params = {i: profit_with_params_dict[i] for i in profit_keys_lst_not_null[:1]}

## Creating the label column (= prediction column)

In [ ]:
# extract best "model" and "label" thresholds as well as the best chosen model parameters from dictionary
label_thresh = list(best_rf_params.values())[0][0]
model_thresh = list(best_rf_params.values())[0][1]
final_rf_params = list(best_rf_params.values())[0][2]

In [ ]:
# create the label column (our "y" column, binary result according to our "label_threshold"% threshold) based on the price difference between 3 months
final_joined_dataframe['label'] = (final_joined_dataframe["diff_in_mc_perc"] > label_thresh).astype(int)

In [ ]:
# check for Nan values in the dataframe
final_joined_dataframe[final_joined_dataframe.isna().any(axis=1)]

In [ ]:
# Convert "date_plus_3m" columns type from 'string' to 'datetime'
final_joined_dataframe['date_plus_3m'] = pd.to_datetime(final_joined_dataframe['date_plus_3m'])

In [ ]:
final_joined_dataframe.shape

# 2. First Option: Running the Random Forest model with best parameters on the test set

## Splitting and training the Data

In [ ]:
# train: 80%, test: 20%
train = final_joined_dataframe.query('"2013-01-01" < date <= "2018-08-29"').copy()
test = final_joined_dataframe.query('"2018-08-29" < date <= "2020-02-01"').copy()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
# Set the x (=features) and y (=label) columns
X_Cols = ['enterprise_val','pe_ratio','pb_ratio','trailing_peg_1_y', 'pfree_cash_flow_ratio', 'graham_value', 'currentRatio']
Y_Cols = ['label']

# Split X and y into X_
X_train, X_test = train[X_Cols], test[X_Cols]
y_train, y_test = train[Y_Cols].values.ravel(), test[Y_Cols].values.ravel()

# Create a Random Forest Classifier
rand_frst_clf = RandomForestClassifier(**final_rf_params)

# Fit (train) the data to the model
rand_frst_clf.fit(X_train, y_train)

# Make predictions (test)
y_pred_proba = rand_frst_clf.predict_proba(X_test)[:,1]
y_pred = y_pred_proba > model_thresh

y_train.shape

In [ ]:
# Print the Accuracy of the Model.
print('Correct Prediction (%): ', accuracy_score(y_test, rand_frst_clf.predict(X_test), normalize = True) * 100.0)

# Compute model report

In [ ]:
# Define the target names
target_names = ['Down Day', 'Up Day']

# Build a classification report
report = classification_report(y_true = y_test, y_pred = y_pred, target_names = target_names, output_dict = True)

# Add it to a data frame, transpose it for readability.
report_df = pd.DataFrame(report).transpose()
report_df

In [ ]:
def present_scores(y_true, y_pred, n_bins = 5, normalized_sizes = True):
    
    """
       A whole package for computing a various of metrics for binary classification, 
       It includes- f1 score for each class, PayPal coverage-precision curve, and roc-auc.


        Parameters
        ----------
        y_true : 1d array-like, or label indicator array / sparse matrix
            Ground truth (correct) target values.

        y_pred : 1d array-like, or label indicator array / sparse matrix
            Estimated targets probas for the positive class as returned by a classifier.

        n_bins : int, optional.
            The number of bins for vizualization PayPal coverage-precision curve. default is 5.

        normalized_sizes : Boolean, optional.
            Whether the axis of coverage will be in absolute numbers or not.
            

        Returns
        -------
        None, just printing and plotting.

    """    
    
    df = pd.DataFrame({"predicted": y_pred, 
                       "actual" : y_true})

    df["prediction_group"] = pd.cut(df["predicted"], bins = np.linspace(0, 1, n_bins +1), include_lowest=True)
    grouped = df.groupby("prediction_group").size()
    if normalized_sizes:
        grouped = grouped/len(df)
    
    ax1 = grouped.plot(kind="bar", color = "blue")
    ax2 = ax1.twinx()
    ax2 = df.groupby("prediction_group")["actual"].mean().plot(color = "orange")
    ax2.axhline(y=y_true.mean(), xmin=0, xmax=1)
    plt.show()
    print(classification_report(y_true, y_pred > model_thresh))
    print("AUC Score:", roc_auc_score(y_true, y_pred))

In [ ]:
print(model_thresh, label_thresh)

In [ ]:
present_scores(y_test, y_pred_proba, n_bins=10)

# Compute confusion matrix

In [ ]:
# Compute a confusion matrix to evaluate the accuracy of the classification - displaying actual total values numbers per category
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=target_names)
disp.plot(cmap=plt.cm.Blues)

plt.show()

# Compute confusion matrix to evaluate the accuracy of the classification - displaying normalized values numbers
cm = confusion_matrix(y_test, y_pred, normalize='all')
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=target_names)
disp.plot(cmap=plt.cm.Blues)

plt.show()



In [ ]:

# הבר (המקל) הכחול מתאר כמה תצפיות יש בכל בין של הסתברות, למשל, בבין של 0.3 עד 0.4 יש 38% מהתצפיות
# הקו הכחול לעומתו, מתאר את המציאות - כמה ״1״-ים יש לנו במציאות
# הקו הכתום, תמיד צריך להיות משווה לקו הכחול, מעיד על אחוז הדיוק בבין
# נשים את הסף כל עוד הקו הכתום מעל הקו הכחול, או במינימום בנק׳ המפגש שלהם
# במודל טוב - ככל שההתסברות גבוהה יותר, כך הוא גם צודק יותר, ולכן בגדול הקו הכתום צריך לעלות, אבל זה לא המצב אצלנו

In [ ]:
#create the profit function - based on the percentage of change in the "market cap percentage" column

# create prediction column in the "test" dataframe based on the model results, set it to binary number instead of True / False
test['prediction'] = y_pred.astype(int)
# test = test.reset_index()
test['profit_percentage'] = test['diff_in_mc_perc'] * test['prediction']
profit_perc_sum = round(test['profit_percentage'].sum(), 3)
profit_perc_mean = round(profit_perc_sum / test[test['prediction']==1].shape[0], 3)

print("The total profit in percentage after investing in tested stock is:",profit_perc_mean,"%")

# 3. Second Option: Running the Random Forest model with best parameters on validation set

## Splitting and training the Data

In [ ]:
train = final_joined_dataframe.query('"2013-01-01" < date <= "2017-03-30"').copy()
validation = final_joined_dataframe.query('"2017-03-30" < date <= "2018-08-29"').copy()
# test = final_joined_dataframe.query('"2018-08-29" < date <= "2020-02-01"').copy()

In [ ]:
train.shape

In [ ]:
validation.shape

In [ ]:
# Set the x (=features) and y (=label) columns
X_Cols = ['enterprise_val','pe_ratio','pb_ratio','trailing_peg_1_y', 'pfree_cash_flow_ratio', 'graham_value', 'currentRatio']
Y_Cols = ['label']

# Split X and y into X_
X_train, X_validation = train[X_Cols], validation[X_Cols]
y_train, y_validation = train[Y_Cols].values.ravel(), validation[Y_Cols].values.ravel()

# Create a Random Forest Classifier
rand_frst_clf = RandomForestClassifier(max_depth= 5, max_features= 8, n_estimators = 151, criterion = "gini", random_state = 711)
# rand_frst_clf = RandomForestClassifier(**final_rf_params)

# Fit (train) the data to the model
rand_frst_clf.fit(X_train, y_train)

# Make predictions (test)
y_pred_proba = rand_frst_clf.predict_proba(X_validation)[:,1]
# y_pred = rand_frst_clf.predict(X_test)
y_pred = y_pred_proba > model_thresh

y_train.shape


In [ ]:
# Print the Accuracy of the Model.
print('Correct Prediction (%): ', accuracy_score(y_test, rand_frst_clf.predict(X_test), normalize = True) * 100.0)

# Compute model report

In [ ]:
# Define the target names
target_names = ['Down Day', 'Up Day']

# Build a classification report
report = classification_report(y_true = y_validation, y_pred = y_pred, target_names = target_names, output_dict = True)

# Add it to a data frame, transpose it for readability.
report_df = pd.DataFrame(report).transpose()
report_df

In [ ]:
def present_scores(y_true, y_pred, n_bins = 5, normalized_sizes = True):

    """
       A whole package for computing a various of metrics for binary classification,
       It includes- f1 score for each class, PayPal coverage-precision curve, and roc-auc.


        Parameters
        ----------
        y_true : 1d array-like, or label indicator array / sparse matrix
            Ground truth (correct) target values.

        y_pred : 1d array-like, or label indicator array / sparse matrix
            Estimated targets probas for the positive class as returned by a classifier.

        n_bins : int, optional.
            The number of bins for vizualization PayPal coverage-precision curve. default is 5.

        normalized_sizes : Boolean, optional.
            Whether the axis of coverage will be in absolute numbers or not.


        Returns
        -------
        None, just printing and plotting.

    """

    df = pd.DataFrame({"predicted": y_pred,
                       "actual" : y_true})

    df["prediction_group"] = pd.cut(df["predicted"], bins = np.linspace(0, 1, n_bins +1), include_lowest=True)
    grouped = df.groupby("prediction_group").size()
    if normalized_sizes:
        grouped = grouped/len(df)

    ax1 = grouped.plot(kind="bar", color = "blue")
    ax2 = ax1.twinx()
    ax2 = df.groupby("prediction_group")["actual"].mean().plot(color = "orange")
    ax2.axhline(y=y_true.mean(), xmin=0, xmax=1)
    plt.show()
    print(classification_report(y_true, y_pred > model_thresh))
    print("AUC Score:", roc_auc_score(y_true, y_pred))

In [ ]:
print(model_thresh, label_thresh)

In [ ]:
present_scores(y_validation, y_pred_proba, n_bins=10)

# Compute confusion matrix

In [ ]:
# Compute a confusion matrix to evaluate the accuracy of the classification - displaying actual total values numbers per category
cm = confusion_matrix(y_validation, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=target_names)
disp.plot(cmap=plt.cm.Blues)

plt.show()

# Compute confusion matrix to evaluate the accuracy of the classification - displaying normalized values numbers
cm = confusion_matrix(y_validation, y_pred, normalize='all')
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=target_names)
disp.plot(cmap=plt.cm.Blues)

plt.show()



In [ ]:
#create the profit function - based on the percentage of change in the "market cap percentage" column

# create prediction column in the "validation" dataframe based on the model results, set it to binary number instead of True / False
validation['prediction'] = y_pred.astype(int)
# test = test.reset_index()
validation['profit_percentage'] = validation['diff_in_mc_perc'] * validation['prediction']
profit_perc_sum = round(test['profit_percentage'].sum(), 3)
profit_perc_mean = round(profit_perc_sum / test[test['prediction']==1].shape[0], 3)

print("The total profit in percentage after investing in tested stock is:",profit_perc_mean,"%")